<h1> Welcome to Extensive US Youtube [EDA] kernel. </h1>


<i>English is not my first language, so sorry about any error.</i>

I will analyse the youtube data to undersand the US videos patterns.

We will find for some informations like:

- What's the most viewed type and category?
- The metrics stats have the same distribuition to all category's?
- Whats the most frequent words in titles, descriptions, tags? 
- Which type of category has more comments?
- And a lot of another interesting questions that we will find

Give me your feedback and if you like my kernel <b>votes up</b> <br>
# NOTE: If you want to see the complete code Fork this kernel

<h2>Importing librarys and data</h2>

In [ ]:
# To manipulate data, NLP and visualizations
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS

# Standard plotly imports
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks

# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
cufflinks.go_offline(connected=True)

# To interactive buttons
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_yout = pd.read_csv("data/USvideos.csv", nrows=15000)

## Shape and Unique values

In [ ]:
def resume_data(dataset):
    '''
    Will return some summary's about the data
    '''
    print("The dataset have the Shape of: ", dataset.shape)
    print("\nUnique values for each Column: ")
    print(dataset.nunique())

In [ ]:
# Looking for the first informations about our data
resume_data(df_yout)

In total the dataset has almost 41k rows and 16 columns. <br>
Based on the output we can infer that our dataset have few categorical features. 


Let's keep investigating what is the data.

## Missing values

Missing values are a frequent problem. <br>
Now we will see if our data contains any missing value. 

In [ ]:
#Looking for Null values and the types of our data
df_yout.info()

Cool, we don't have missing values at the data. <br>

The data types seems correct except for trending_date and publish_time. Further, we will explore more this columns and apply transformations if necessary.

## Knowing our data

It's important to see how the data is. So we will plot few lines to understanding what are we 

In [ ]:
df_yout.head()

<h2>Let's start looking if Views, likes, dislikes and comment_counts have a normal distribuition</h2>

- First, I will transform the metrics in log distribuitions to see the distribuition of them.

In [ ]:
# Creating the log of metrics
df_yout['likes_log'] = np.log(df_yout['likes'] + 1)
df_yout['views_log'] = np.log(df_yout['views'] + 1)
df_yout['dislikes_log'] = np.log(df_yout['dislikes'] + 1)
df_yout['comment_log'] = np.log(df_yout['comment_count'] + 1)

# df_yout['views_like_ratio'] = df_yout['views'] / df_yout['likes']
# df_yout['views_dislike_ratio'] = df_yout['views'] / df_yout['dislikes']
# df_yout['views_comments_ratio'] = df_yout['views'] / df_yout['comment_count']
# df_yout['dislike_comments_ratio'] = df_yout['dislikes'] / df_yout['comment_count']
# df_yout['likes_comments_ratio'] = df_yout['views'] / df_yout['comment_count']

### Ploting the log distribuitions

In [ ]:
df_yout[['views_log', 'likes_log','dislikes_log', 'comment_log']].iplot(kind="histogram", 
                                                                        bins=50, theme="white", 
                                                                        histnorm='probability', 
title="Log Distribuitions of Metrics",
                                                                        xTitle='Logarithmn Distribuition',
                                                                        yTitle='Probability')

Very interesting distribuition of our metrics. 

We can see that Dislikes and Comments has similar values (Maybe the people comment when dislikes the videos?). <br>
Important to note, that on comment distribuition, we have a high number of videos without comments.<br>
The views distribuition has the highest values, which make total sense and likes with the peak of dist at 9.5 ~ 10.5;


## Let's invetigate the quantiles and Outliers in our data

In [ ]:
def quantiles(columns):
    for name in columns:
        print(name + " quantiles")
        print(df_yout[name].quantile([.01,.25,.5,.75,.99]))
        print("")

In [ ]:
quantiles(['views','likes', 'dislikes', 'comment_count'])

### Outliers

In [ ]:
def CalcOutliers(df_num): 

    # calculating mean and std of the array
    data_mean, data_std = np.mean(df_num), np.std(df_num)

    # seting the cut line to both higher and lower values
    # You can change this value
    cut = data_std * 3

    #Calculating the higher and lower cut values
    lower, upper = data_mean - cut, data_mean + cut

    # creating an array of lower, higher and total outlier values 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = [x for x in df_num if x < lower or x > upper]

    # array without outlier values
    outliers_removed = [x for x in df_num if x > lower and x < upper]
  
    print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
    print('Identified outliers: %d' % len(outliers_total)) # printing total number of values outliers of both sides
    print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) # Percentual of outliers in points
    print("\n")  
    
    return

In [ ]:
## Instantianting the function to detect outliers
[(print("Total Outliers at " + col), CalcOutliers(df_yout[col]), ) for col in ['views', 'likes', 'dislikes', 'comment_count']]

## Click on Show output to see the results

Interesting! All the outliers is about high values and the maximum outliers values is 1.44% in likes;

## Renaming the categorys id to names

In [ ]:
rename_channels = {1:'Film/Animation', 2:'Cars/Vehicles', 10:'Music', 15:'Pets/Animals',
                   17:'Sport',19:'Travel/Events',20: 'Gaming',22:'People/Blogs',23:'Comedy',
                   24:'Entertainment',25:'News/Politics',26:'How to/Style',27:'Education',
                   28:'Science/Technology',29:'Non Profits/Activism'}

In [ ]:
df_yout['category_name'] = np.nan

df_yout['category_name'] = df_yout['category_id'].map(rename_channels)

## Let's use another chart to see the proporcionality of each category in our dataset.

In [ ]:
percent_category = round(df_yout["category_name"].value_counts(), 2)

# print("Category percentual: ")
## print(round(percent_category[:5] / len(df_yout["category_name"]) * 100,2))

types = round(df_yout["category_name"].value_counts() / len(df_yout["category_name"]) * 100,2)

labels = list(types.index)
values = list(types.values)

trace1 = go.Pie(labels=labels, 
                values=values, 
                marker=dict(colors=['red']), 
                # Seting values to 
                text=(percent_category.index.map(str)),
                hovertext=(percent_category.values.astype(str)))

layout = go.Layout(title="Percentual of each Category in upload ", 
                   legend=dict(orientation="h"));

fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)


We can see that the most frequent video categorys are: <br>
1 - Entertainment  <br>
2 - Music <br>
3 - How to and Style 

<h2>Looking the Views, likes, dislikes distribuition by category_name's</h2>

In [ ]:
## Building a 
trace1 = go.Box(x=df_yout['category_name'],
                y=df_yout['views_log'], name='Views Log',visible=True, boxpoints='outliers')

trace2 = go.Box(x=df_yout['category_name'],
                y=df_yout['likes_log'], name='Likes Log', visible=False,boxpoints='outliers')

trace3 = go.Box(x=df_yout['category_name'], 
                y=df_yout['dislikes_log'], name='Dislikes Log', visible=False,boxpoints='outliers')

trace4 = go.Box(x=df_yout['category_name'],
                y=df_yout['comment_log'], name='Comment Log', visible=False,boxpoints='outliers')

data = [trace1, trace2, trace3, trace4]

updatemenus = list([
    dict(active=-1,
         showactive=True,
         buttons=list([  
            dict(
                label = 'Views by Categorys',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]}, 
                     {'title': 'View Logs by each Category'}]),
             
             dict(
                  label = 'Likes by Categorys',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                     {'title': 'Likes Logs by each Category'}]),

            dict(
                 label = 'Dislikes by Categorys',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                     {'title': 'Dislikes Logs by each Category'}]),

            dict(
                 label = 'Comments by Categorys',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                     {'title': 'Comments Logs by each Category'}])
        ]),
    )
])


layout = dict(title='## (Select from Dropdown) ## - The Log Distribuition of Metrics by video Category <br> View Logs by each Category', 
              showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)

iplot(fig)

## Interactive Plotly to pick the desired metric
- Total views by category
- Total Unique channels in each category
- Total Unique videos in each category


In [ ]:
trace1 = go.Bar(x=df_yout.groupby(['category_name'])['views'].sum().index,
                y=df_yout.groupby(['category_name'])['views'].sum().values,
                name='Total Views by Category', visible=True)

trace2 = go.Bar(x=df_yout.groupby(['category_name'])['channel_title'].nunique().index,
                y=df_yout.groupby(['category_name'])['channel_title'].nunique().values, 
                name='Unique Channels by Category', visible=False)

trace3 = go.Bar(x=df_yout.groupby(['category_name'])['title'].nunique().index,
                y=df_yout.groupby(['category_name'])['title'].nunique().values, 
                name='Unique Videos by Category', visible=False)


data = [trace1, trace2, trace3]

updatemenus = list([
    dict(active=-1,
         showactive=True,
         buttons=list([  
            dict(
                label = 'Total Views by Category',
                 method = 'update',
                 args = [{'visible': [True, False, False]}, 
                     {'title': 'Sum of views by Category'}]),
             
             dict(
                  label = 'Total Channels by Category',
                 method = 'update',
                 args = [{'visible': [False, True, False]},
                     {'title': 'Total Channels by Category'}]),

            dict(
                 label = 'Total Videos by Categorys',
                 method = 'update',
                 args = [{'visible': [False, False, True]},
                     {'title': 'Total Videos by Category'}])
        ]),
    )
])


layout = dict(title='*(Select from Dropdown)* Different Metrics to each Category <br> Sum of views by Category', 
              showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)

iplot(fig)

## The top 25 channels with more views

In [ ]:
df_yout.groupby(['channel_title'])['views'].sum().nlargest(25).iplot(kind='bar',
                                                                     xTitle='Channel Name', yTitle='Total Views',
                                                                     title='TOP 25 Channels with more views')

## The top 25 channels with more videos in youtube

In [ ]:
df_yout.groupby(['channel_title'])['title'].nunique().nlargest(25).iplot(kind='bar',
                                                             xTitle='Channel Names', yTitle='Total Videos Uploaded',
                                                             title='TOP 25 channels with more uploaded videos')

<h1>Engagement Features </h1>

I will create some new variables to us analyse the percent of likes, dislikes, comment by each category to discover what category have the highest engagement

In [ ]:
df_yout['like_rate'] =  df_yout ['likes'] / df_yout['views'] * 100
df_yout['dislike_rate'] =  df_yout ['dislikes'] / df_yout['views'] * 100
df_yout['comment_rate'] =  df_yout ['comment_count'] / df_yout['views'] * 100

Very meaningful distribuition! Like x Dislikes rate have interesting differences

In [ ]:
## Building a 
trace1 = go.Box(x=df_yout['category_name'],
                y=df_yout['like_rate'], name='Views Log',visible=True, boxpoints='outliers')

trace2 = go.Box(x=df_yout['category_name'],
                y=df_yout['dislike_rate'], name='Likes Log', boxpoints='outliers')

trace3 = go.Box(x=df_yout['category_name'], 
                y=df_yout['comment_rate'], name='Dislikes Log', boxpoints='outliers')

data = [trace1, trace2, trace3]

updatemenus = list([
    dict(active=-1,
         showactive=True,
         buttons=list([  
            dict(
                label = 'Like rate',
                 method = 'update',
                 args = [{'visible': [True, False, False]}, 
                     {'title': 'Like Rate by Category'}]),
             
             dict(
                  label = 'Dislike Rate',
                 method = 'update',
                 args = [{'visible': [False, True, False]},
                     {'title': 'Dislikes Rate by Category'}]),

            dict(
                 label = 'Comment Rate',
                 method = 'update',
                 args = [{'visible': [False, False, True]},
                     {'title': 'Total Comments Rate by Category'}])
        ]),
    )
])


layout = dict(title="The Rate of Metrics to all video Category's (Select from Dropdown)", 
              showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)

iplot(fig)

We can see that the Dislike rate is very low in almost all categories but some outliers in 'People and Blogs' and News & Politics that we can might can consider "Normal"

The mean of ike distribuition is less than 5% but in music we have a very interesting pattern of like rate... The music category have the highest engagement rate.

And at the Comment rate we can see the how-to category with the highest rates of commments. Interesting because isn't politics LOL

<h2>Looking through the anothers categoricals variables </h2>
We have the variables:
- Comments Disable
- Rating disable
- Video error


In [ ]:

#First plot
trace0 = go.Histogram(
    x=df_yout['comments_disabled'], histnorm='percent', marker=dict( color='blue' ),
    showlegend=False
)
#Second plot
trace1 = go.Histogram(
    x=df_yout['ratings_disabled'], histnorm='percent', marker=dict( color='blue' ), 
    showlegend=False
)
#Third plot
trace2 = go.Histogram(
    x=df_yout['video_error_or_removed'], histnorm='percent', marker=dict( color='blue' ), 
    showlegend=False
)
trace3 = go.Box(
    x=df_yout['comments_disabled'], y=df_yout['views_log'], marker=dict( color='blue' ), 
    showlegend=False
)
#Second plot
trace4 = go.Box(
    x=df_yout['ratings_disabled'], y=df_yout['views_log'], marker=dict( color='blue' ), 
    showlegend=False
)   
#Third plot
trace5 = go.Box(
    x=df_yout['video_error_or_removed'], y=df_yout['views_log'],  marker=dict( color='blue' ), 
    showlegend=False
)

#Creating the grid
fig = tls.make_subplots(rows=2, cols=3, specs=[[{}, {}, {}], [{}, {}, {}]],
                          subplot_titles=("% Comment Disabled",
                                          "% Likes/Dislikes Disabled",
                                          "% Video Error/Removed",
                                          "Views Comment Disabled",
                                          "Views Likes/Dislikes Disabled",
                                          "% Video Error/Removed"
                                          ))
#setting the figs
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 3)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 3)
fig.append_trace(trace5, 2, 2)

fig['layout'].update(showlegend=True, title="Disabled or Error Metrics of Videos")
iplot(fig)

The highest 

<h2>Let's look the kde of this variables separated to deep understand them</h2>

In [ ]:
df_yout[df_yout['video_error_or_removed'] == True ]['views_log']
#df_yout[df_yout['ratings_disabled'] == True]
type(df_yout[df_yout['comments_disabled'] == True]['views_log'])
#type(df_yout[df_yout['video_error_or_removed'] == True]['comment_log'])

In [ ]:
plt.figure(figsize = (12,10))

plt.subplot(221)
g1 = sns.distplot(df_yout[df_yout['comments_disabled'] == True]['views_log'], 
                  hist=False, label='Comm_dis')
g1 = sns.distplot(df_yout[df_yout['ratings_disabled'] == True]['views_log'], 
                  hist=False, label='Rati_dis')
#g1 = sns.distplot(df_yout[df_yout['video_error_or_removed'] == True]['views_log'], 
#                  hist=False, label='vide_rmv_err')
g1 = sns.distplot(df_yout['views_log'], 
                  hist=False, label='Total')
g1.set_title("VIEWS LOG DISTRIBUITION", fontsize=16)

plt.subplot(222)
g4 = sns.distplot(df_yout[df_yout['ratings_disabled'] == True]['comment_log'], 
                  hist=False, label='Rati_dis')
#g4 = sns.distplot(df_yout[df_yout['video_error_or_removed'] == True]['comment_log'], 
#                  hist=False, label='vide_rmv_err')
g4 = sns.distplot(df_yout['comment_log'], 
                  hist=False, label='Total')
g4.set_title("COMMENTS LOG DISTRIBUITION", fontsize=16)

plt.subplot(223)
g3 = sns.distplot(df_yout[df_yout['comments_disabled'] == True]['dislikes_log'], 
                  hist=False, label='Comm_dis')
#g3 = sns.distplot(df_yout[df_yout['video_error_or_removed'] == True]['dislikes_log'], 
#                  hist=False, label='vide_rmv_err')
g3 = sns.distplot(df_yout['dislikes_log'], 
                  hist=False, label='Total')
g3.set_title("DISLIKES LOG DISTRIBUITION", fontsize=16)

plt.subplot(224)
g2 = sns.distplot(df_yout[df_yout['comments_disabled'] == True]['likes_log'], 
                  hist=False, label='Comm_dis')
#g2 = sns.distplot(df_yout[df_yout['video_error_or_removed'] == True]['likes_log'], 
#                  hist=False, label='vide_rmv_err')
g2 = sns.distplot(df_yout['likes_log'], 
                  hist=False, label='Total')
g2.set_title('LIKES LOG DISTRIBUITION', fontsize=16)

plt.subplots_adjust(wspace = 0.2, hspace = 0.3,top = 0.9)
plt.legend()
plt.show()

Removed videos no have some views until be moreved... <br>Rating disable and Comments disable have the same distribuition 

<h2>Analysing the Correlation Matrix of the numerical variables</h2>

<h2>I will start creating some features from texts using Title and Tags columns</h2>

In [ ]:
#nlp
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer   

In [ ]:
 #nltk.download()

In [ ]:
#Setting the stopwords
eng_stopwords = set(stopwords.words("english"))


In [ ]:
#Word count in each comment:
df_yout['count_word']=df_yout["title"].apply(lambda x: len(str(x).split()))
df_yout['count_word_tags']=df_yout["tags"].apply(lambda x: len(str(x).split()))

#Unique word count
df_yout['count_unique_word']=df_yout["title"].apply(lambda x: len(set(str(x).split())))
df_yout['count_unique_word_tags']=df_yout["tags"].apply(lambda x: len(set(str(x).split())))

#Letter count
df_yout['count_letters']=df_yout["title"].apply(lambda x: len(str(x)))
df_yout['count_letters_tags']=df_yout["tags"].apply(lambda x: len(str(x)))

#punctuation count
df_yout["count_punctuations"] =df_yout["title"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_yout["count_punctuations_tags"] =df_yout["tags"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

#upper case words count
df_yout["count_words_upper"] = df_yout["title"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_yout["count_words_upper_tags"] = df_yout["tags"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

#title case words count
df_yout["count_words_title"] = df_yout["title"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_yout["count_words_title_tags"] = df_yout["tags"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

#Number of stopwords
df_yout["count_stopwords"] = df_yout["title"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_yout["count_stopwords_tags"] = df_yout["tags"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

#Average length of the words
df_yout["mean_word_len"] = df_yout["title"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_yout["mean_word_len_tags"] = df_yout["tags"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
#derived features
#Word count percent in each comment:
df_yout['word_unique_percent']=df_yout['count_unique_word']*100/df_yout['count_word']
df_yout['word_unique_percent_tags']=df_yout['count_unique_word_tags']*100/df_yout['count_word_tags']

#Punct percent in each comment:
df_yout['punct_percent']=df_yout['count_punctuations']*100/df_yout['count_word']
df_yout['punct_percent_tags']=df_yout['count_punctuations_tags']*100/df_yout['count_word_tags']

In [ ]:
plt.figure(figsize = (12,18))

plt.subplot(421)
g1 = sns.distplot(df_yout['count_word'], 
                  hist=False, label='Text')
g1 = sns.distplot(df_yout['count_word_tags'], 
                  hist=False, label='Tags')
g1.set_title("COUNT WORDS DISTRIBUITION", fontsize=16)

plt.subplot(422)
g2 = sns.distplot(df_yout['count_unique_word'],
                  hist=False, label='Text')
g2 = sns.distplot(df_yout['count_unique_word_tags'], 
                  hist=False, label='Tags')
g2.set_title("COUNT UNIQUE DISTRIBUITION", fontsize=16)

plt.subplot(423)
g3 = sns.distplot(df_yout['count_letters'], 
                  hist=False, label='Text')
g3 = sns.distplot(df_yout['count_letters_tags'], 
                  hist=False, label='Tags')
g3.set_title("COUNT LETTERS DISTRIBUITION", fontsize=16)

plt.subplot(424)
g4 = sns.distplot(df_yout["count_punctuations"], 
                  hist=False, label='Text')
g4 = sns.distplot(df_yout["count_punctuations_tags"], 
                  hist=False, label='Tags')
g4.set_xlim([-2,50])
g4.set_title('COUNT PONCTUATIONS DISTRIBUITION', fontsize=16)

plt.subplot(425)
g5 = sns.distplot(df_yout["count_words_upper"] , 
                  hist=False, label='Text')
g5 = sns.distplot(df_yout["count_words_upper_tags"] , 
                  hist=False, label='Tags')
g5.set_title('COUNT WORDS UPPER DISTRIBUITION', fontsize=16)

plt.subplot(426)
g6 = sns.distplot(df_yout["count_words_title"], 
                  hist=False, label='Text')
g6 = sns.distplot(df_yout["count_words_title_tags"], 
                  hist=False, label='Tags')
g6.set_title('WORDS DISTRIBUITION', fontsize=16)

plt.subplot(427)
g7 = sns.distplot(df_yout["count_stopwords"], 
                  hist=False, label='Title')
g7 = sns.distplot(df_yout["count_stopwords_tags"], 
                  hist=False, label='Tags')
g7.set_title('STOPWORDS DISTRIBUITION', fontsize=16)

plt.subplot(428)
g8 = sns.distplot(df_yout["mean_word_len"], 
                  hist=False, label='Text')
g8 = sns.distplot(df_yout["mean_word_len_tags"], 
                  hist=False, label='Tags')
g8.set_xlim([-2,100])
g8.set_title('MEAN WORD LEN DISTRIBUITION', fontsize=16)

plt.subplots_adjust(wspace = 0.2, hspace = 0.4,top = 0.9)
plt.legend()
plt.show()

<h2>Have the Punctuations title and tags any relation with views, likes, dislikes comments? </h2>

In [ ]:
trace1 = go.Box(x=df_yout['count_punctuations'],
                y=df_yout['views_log'], name='Views Log',visible=True, 
                boxpoints='outliers')

trace2 = go.Box(x=df_yout['count_punctuations'],
                y=df_yout['likes_log'], name='Likes Log', 
                boxpoints='outliers')

trace3 = go.Box(x=df_yout['count_punctuations'], 
                y=df_yout['dislikes_log'], name='Dislikes Log', 
                boxpoints='outliers')

trace4 = go.Box(x=df_yout['count_punctuations'],
                y=df_yout['comment_log'], name='Comment Log',
                boxpoints='outliers')

data = [trace1, trace2, trace3, trace4]

updatemenus = list([
    dict(active=-1,
         showactive=True,
         buttons=list([  
            dict(
                label = 'Views by Total Punctuation',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]}, 
                     {'title': 'View Logs by Total Punctuation'}]),
             
             dict(
                  label = 'Likes by Total Punctuation',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                     {'title': 'Likes Logs by Total Punctuation'}]),

            dict(
                 label = 'Dislikes by Total Punctuation',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                     {'title': 'Dislikes Logs by Total Punctuation'}]),

            dict(
                 label = 'Comments by Total Punctuation',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                     {'title': 'Comments Logs by Total Punctuation'}])
        ]),
    )
])


layout = dict(title='The Log Distribuition of Metrics by Punctuation (Select from Dropdown)', 
              showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)

iplot(fig)

<h2>The same view using Tags punctuations values</h2>

In [ ]:
trace1 = go.Box(x=df_yout[df_yout['count_punctuations_tags'] < 100]['count_punctuations_tags'],
                y=df_yout[df_yout['count_punctuations_tags'] < 100]['views_log'], name='Views Log',visible=True, boxpoints='outliers')

trace2 = go.Box(x=df_yout[df_yout['count_punctuations_tags'] < 100]['count_punctuations_tags'],
                y=df_yout[df_yout['count_punctuations_tags'] < 100]['likes_log'], name='Likes Log', boxpoints='outliers')

trace3 = go.Box(x=df_yout[df_yout['count_punctuations_tags'] < 100]['count_punctuations_tags'], 
                y=df_yout[df_yout['count_punctuations_tags'] < 100]['dislikes_log'], name='Dislikes Log', boxpoints='outliers')

trace4 = go.Box(x=df_yout[df_yout['count_punctuations_tags'] < 100]['count_punctuations_tags'],
                y=df_yout[df_yout['count_punctuations_tags'] < 100]['comment_log'], name='Comment Log', boxpoints='outliers')

data = [trace1, trace2, trace3, trace4]

updatemenus = list([
    dict(active=-1,
         showactive=True,
         buttons=list([  
            dict(
                label = 'Views by Tags count',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]}, 
                     {'title': 'Vews by Ponctuations tags'}]),
             
             dict(
                  label = 'Likes by Tags count',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                     {'title': 'Likes by Ponctuations tags'}]),

            dict(
                 label = 'Dislikes by Tags count',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                     {'title': 'Dislikes by Ponctuations tags'}]),

            dict(
                 label = 'Comments by Tags count',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                     {'title': 'Comments by Ponctuations tags'}])
        ]),
    )
])


layout = dict(title='The Log Distribuition of Metrics to all Tags (Select from Dropdown)', 
              showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)

iplot(fig)

<h2>Let's verify the correlation of title and tags values with views</h2>
- exploring the Correlation matrix to see if have any interesting relation

In [ ]:
plt.figure(figsize = (14,10))

sns.heatmap(df_yout[['count_word', 'count_unique_word','count_letters',
                     "count_punctuations","count_words_upper", "count_words_title", 
                     "count_stopwords","mean_word_len", 
                     'views_log', 'likes_log','dislikes_log','comment_log',
                     'ratings_disabled', 'comments_disabled', 'video_error_or_removed']].corr(), annot=True)
plt.show()

<h1>Visualizing  the Word Cloud of Tagged videos </h1>
- Ratings Disable 
- Comments Disable
- Video Error or Removed

In [ ]:
plt.rcParams['font.size']= 15              
plt.rcParams['savefig.dpi']= 100         
plt.rcParams['figure.subplot.bottom']= .1 

<h3>Rating Disabled</h3>

In [ ]:
plt.figure(figsize = (15,15))

stopwords = set(STOPWORDS)

wordcloud = WordCloud(
                          background_color='black',
                          stopwords=stopwords,
                          max_words=1000,
                          max_font_size=120, 
                          random_state=42
                         ).generate(str(df_yout['title']))

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.title("WORD CLOUD - TITLES")
plt.axis('off')
plt.show()

<h3>Comments Disabled</h3>

<h2>General Title Word Cloud</h2>

In [ ]:
plt.figure(figsize = (15,15))

stopwords = set(STOPWORDS)
wordcloud = WordCloud(
                          background_color='black',
                          stopwords=stopwords,
                          max_words=150,
                          max_font_size=40, 
                          random_state=42
                         ).generate(str(df_yout['title']))

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.title("WORD CLOUD - DESCRIPTION")
plt.axis('off')
plt.show()

<h1>Visualizing the WordCloud of Description</h1>

In [ ]:
plt.figure(figsize = (15,15))

stopwords = set(STOPWORDS)

newStopWords= ['https', 'youtube', 'VIDEO','youtu','CHANNEL', 'WATCH']

stopwords.update(newStopWords)

wordcloud = WordCloud(
                          background_color='black',
                          stopwords=stopwords,
                          max_words=1200,
                          max_font_size=80, 
                          random_state=42
                         ).generate(str(df_yout['description']))

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.title("WORD CLOUD - DESCRIPTION")
plt.axis('off')
plt.show()


<h1>Analysing the Wordcloud of Tags</h1>

In [ ]:
plt.figure(figsize = (15,15))

stopwords = set(STOPWORDS)
wordcloud = WordCloud(
                          background_color='black',
                          stopwords=stopwords,
                          max_words=1200,
                          max_font_size=120, 
                          random_state=42
                         ).generate(str(df_yout['tags']))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.title("WORD CLOUD - TAGS")
plt.axis('off')
plt.show()

I will continue implementing this analysis because I want learn about text data! <br>

Thank you very much for your view and <b> votes up</b> my kernel 

In [ ]:
df_yout['publish_time'] = pd.to_datetime(df_yout['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

In [ ]:
df_yout['month'] = df_yout['publish_time'].dt.month

print("Category Name count")
print(df_yout['month'].value_counts()[:5])

plt.figure(figsize = (14,9))

plt.subplot(211)
g = sns.countplot('month', data=df_yout, palette="Set1")
g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_title("Counting Months ", fontsize=20)
g.set_xlabel("Months", fontsize=15)
g.set_ylabel("Count", fontsize=15)

plt.subplot(212)
g1 = sns.lvplot(x='month', y='like_rate', data=df_yout, palette="Set1")
g1.set_xticklabels(g.get_xticklabels(),rotation=45)
g1.set_title("Like Rate by Month", fontsize=20)
g1.set_xlabel("Months", fontsize=15)
g1.set_ylabel("Like Rate(log)", fontsize=15)

plt.subplots_adjust(hspace = 0.5, top = 0.9)

plt.show()

## Let's extract the hour value of datetime'

In [ ]:
# separates date and time into two columns from 'publish_time' column
df_yout.insert(4, 'publish_date', df_yout['publish_time'].dt.date)
df_yout['publish_time'] = df_yout['publish_time'].dt.time
df_yout[['hour','min','sec']] = df_yout['publish_time'].astype(str).str.split(':', expand=True).astype(int)

## Knowning the Hour feature

In [ ]:
print("TOP 5 Hour values")
print(df_yout['hour'].value_counts()[:5])

plt.figure(figsize = (14,9))

plt.subplot(211)
g = sns.countplot('hour', data=df_yout, palette="Set1")
g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_title("Counting Hour ", fontsize=20)
g.set_xlabel("Hours", fontsize=15)
g.set_ylabel("Count", fontsize=15)

plt.subplot(212)
g1 = sns.lvplot(x='hour', y='like_rate', data=df_yout, palette="Set1")
g1.set_xticklabels(g.get_xticklabels(),rotation=45)
g1.set_title("Like Rate by Hour", fontsize=20)
g1.set_xlabel("Hours", fontsize=15)
g1.set_ylabel("Like Rate(%)", fontsize=15)

plt.subplots_adjust(hspace = 0.5, top = 0.9)

plt.show()

In [ ]:
plt.figure(figsize=(14,5))

g1 = sns.lvplot(x='hour', y='comment_rate', data=df_yout, palette="Set1")
g1.set_xticklabels(g.get_xticklabels(),rotation=45)
g1.set_title("Comment Rate by Hour", fontsize=20)
g1.set_xlabel("Hours", fontsize=15)
g1.set_ylabel("Comment Rate(%)", fontsize=15)

plt.show()

Very interesting distribuition by hour

# NOTE: If you want to see all code, fork this kernel

# Votesup and Stay Tuned because I will continue this analysis

In [ ]:
output_test = widgets.Output()
range_slider = widgets.IntRangeSlider(value=[1, (df_yout['views'].max() -1)], 
                                      min=1, 
                                      max=(df_yout['views'].max() -1),
                                      step=100, description='Range: ')
y_widget = widgets.Dropdown(options=['views','likes'], description='Metrics: ', value='views')

def update_x_range(*args):
    output_test.clear_output()
    if (y_widget.value == 'views'):
        range_slider.value = (1,df_yout['views'].max() -1)
        range_slider.max = (df_yout['views'].max() -1)
        
    elif (y_widget.value != 'views'):
        range_slider.value = (1, df_yout['likes'].max() -1)
        range_slider.max = (df_yout['likes'].max() -1)
    else:
        pass
    
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)